<a href="https://colab.research.google.com/github/osagieomigie/TensorFlow_ML/blob/main/coronary_heart_disease/coronaryHeartDisease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an exercise based on the tutorial found here: https://www.tensorflow.org/tutorials/load_data/csv 

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [6]:
!pip install -q sklearn

In [165]:
import pandas as pd
import numpy as np
import io

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers, regularizers
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split

In [69]:
# Upload heart csv file
from google.colab import files
uploaded = files.upload()

Saving heart_test.csv to heart_test (1).csv
Saving heart_train.csv to heart_train (1).csv


In [84]:
# read file
heart_train_data = pd.read_csv(io.BytesIO(uploaded['heart_train.csv']))
heart_test_data = pd.read_csv(io.BytesIO(uploaded['heart_test.csv']))

# visualize
heart_train_data.head()
heart_test_data.head()


,row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,268,118,0.12,4.16,9.37,Absent,57,19.61,0.0,17,0
1,398,162,7.00,7.67,34.34,Present,33,30.77,0.0,62,0
2,408,180,25.01,3.70,38.11,Present,57,30.54,0.0,61,1
3,146,144,0.00,3.84,18.72,Absent,56,22.10,4.8,40,0
4,73,130,5.60,3.37,24.80,Absent,58,25.76,43.2,36,0


In [95]:
#heart_features = heart_data.copy()
# target = heart_data['chd']
# heart_data = heart_data.drop(columns=['chd'])
# #print(heart_data.dtypes)
# #heart_data['famhist'] = heart_data['famhist'].astype(int)

# #print(heart_data.dtypes)

# #chd_label = heart_data.pop('chd')
# #train, test = train_test_split(heart_data, , test_size=0.2)
# x_train, x_test, y_train, y_test = train_test_split(heart_data, target, test_size=0.20, random_state=0)

# # print(heart_data.to_numpy())
# print(len(y_train))
# print(len(y_test))

hTrainFeatures = heart_train_data.copy()
chd_train_labels = hTrainFeatures.pop('chd')
hTestFeatures = heart_test_data.copy()
chd_test_labels = hTestFeatures.pop('chd')

--Split data--


In [96]:
# Create a symbolic input
input = tf.keras.Input(shape=(), dtype=tf.float32)

# Do a calculation using is
result = 2*input + 1

calc = tf.keras.Model(inputs=input, outputs=result)

In [97]:
def build_symbolic_objs(features):
  inputs = {}

  for name, column in features.items():
    dtype = column.dtype
    if dtype == object:
      dtype = tf.string
    else:
      dtype = tf.float32

    inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)
  return inputs 

h_train_inputs = build_symbolic_objs(hTrainFeatures)
h_test_inputs = build_symbolic_objs(hTestFeatures)



In [98]:
#Normalization layer 

def get_all_numeric_inputs(inputs, inputSet):
  numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

  x = layers.Concatenate()(list(numeric_inputs.values()))
  norm = preprocessing.Normalization()
  norm.adapt(np.array(inputSet[numeric_inputs.keys()]))
  all_numeric_inputs = norm(x)
  return all_numeric_inputs

train_all_num_inputs = get_all_numeric_inputs(h_train_inputs, heart_train_data)
test_all_num_inputs = get_all_numeric_inputs(h_test_inputs, heart_test_data)

# Collect all the symbolic preprocessing results, to concatenate them later
train_preprocessed_inputs = [train_all_num_inputs]
test_preprocessed_inputs = [test_all_num_inputs]


In [99]:
# map from strings to integer indices in a vocabulary

def map_strings_to_ints(inputs, preprocessed_inputs, features):
  for name, input in inputs.items():
    if input.dtype == tf.float32:
      continue
    
    lookup = preprocessing.StringLookup(vocabulary=np.unique(features[name]))
    one_hot = preprocessing.CategoryEncoding(max_tokens=lookup.vocab_size())

    x = lookup(input)
    x = one_hot(x)
    preprocessed_inputs.append(x)

map_strings_to_ints(h_train_inputs, train_preprocessed_inputs, hTrainFeatures) 
map_strings_to_ints(h_test_inputs, test_preprocessed_inputs, hTestFeatures) 

In [100]:
train_preprocessed_inputs_cat = layers.Concatenate()(train_preprocessed_inputs)
test_preprocessed_inputs_cat = layers.Concatenate()(test_preprocessed_inputs)

chd_preprocessing = tf.keras.Model(h_train_inputs, train_preprocessed_inputs_cat)
train_features_dict = {name: np.array(value) 
                         for name, value in hTrainFeatures.items()}

test_features_dict = {name: np.array(value) 
                         for name, value in hTestFeatures.items()}

In [173]:
# build model 
def chd_model(preprocessing_head, inputs):
  body = tf.keras.Sequential([
    layers.Dense(256, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),
    layers.Dense(256, activation='elu', kernel_regularizer=regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),
    #layers.Dense(64, activation='relu'),
    #tf.keras.layers.Dropout(0.1),
    layers.Dense(1)
  ])

  preprocessed_inputs = preprocessing_head(inputs)
  result = body(preprocessed_inputs)
  model = tf.keras.Model(inputs, result)

  model.compile(loss=tf.losses.BinaryCrossentropy(from_logits=True),
                optimizer='adam', metrics=['accuracy'])
  return model

chd_model = chd_model(chd_preprocessing, h_train_inputs)

# train model 
chd_model.fit(x=train_features_dict, y=chd_train_labels, epochs=100, verbose=2)

print("--Evaluate model--")
model_loss, model_acc = chd_model.evaluate(x=test_features_dict, y=chd_test_labels, verbose=2)
print(f"Model Loss:    {model_loss:.2f}")
print(f"Model Accuracy: {model_acc*100:.1f}%")

Epoch 1/100
12/12 - 0s - loss: 0.9212 - accuracy: 0.6721
Epoch 2/100
12/12 - 0s - loss: 0.8411 - accuracy: 0.7073
Epoch 3/100
12/12 - 0s - loss: 0.8262 - accuracy: 0.7019
Epoch 4/100
12/12 - 0s - loss: 0.7971 - accuracy: 0.7154
Epoch 5/100
12/12 - 0s - loss: 0.8114 - accuracy: 0.7209
Epoch 6/100
12/12 - 0s - loss: 0.7856 - accuracy: 0.7317
Epoch 7/100
12/12 - 0s - loss: 0.7939 - accuracy: 0.7236
Epoch 8/100
12/12 - 0s - loss: 0.7866 - accuracy: 0.7046
Epoch 9/100
12/12 - 0s - loss: 0.7754 - accuracy: 0.7073
Epoch 10/100
12/12 - 0s - loss: 0.7730 - accuracy: 0.6992
Epoch 11/100
12/12 - 0s - loss: 0.7524 - accuracy: 0.7290
Epoch 12/100
12/12 - 0s - loss: 0.7438 - accuracy: 0.7317
Epoch 13/100
12/12 - 0s - loss: 0.7861 - accuracy: 0.7019
Epoch 14/100
12/12 - 0s - loss: 0.7384 - accuracy: 0.7209
Epoch 15/100
12/12 - 0s - loss: 0.7435 - accuracy: 0.7073
Epoch 16/100
12/12 - 0s - loss: 0.7562 - accuracy: 0.6883
Epoch 17/100
12/12 - 0s - loss: 0.7389 - accuracy: 0.7046
Epoch 18/100
12/12 - 0s